<a href="https://colab.research.google.com/github/Pelmenoff/data_science/blob/main/hw10_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Input, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Nadam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split

In [ ]:
# Завантаження даних
(x, y), _ = fashion_mnist.load_data()

# Спочатку розділяємо дані на 70% тренувальні і 30% для тестування та валідації
x_train, x_test_raw, y_train, y_test_raw = train_test_split(x, y, train_size=0.7, random_state=555)

# Тепер розділимо ці 30% на 20% для тесту і 10% для валідації
# 20% від загального числа = 20/30 частини від x_test_raw та y_test_raw
test_size = 20 / 30
x_val, x_test, y_val, y_test = train_test_split(x_test_raw, y_test_raw, test_size=test_size, random_state=555)

print("Train data size", x_train.shape)
print("Test data size", x_test.shape)
print("Validation data size", x_val.shape)

# Масштабування зображень до діапазону [0, 1]
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Розширення розмірності (додавання каналу)
x_train = np.expand_dims(x_train, -1)
x_val = np.expand_dims(x_val, -1)
x_test = np.expand_dims(x_test, -1)

# One-hot кодування міток
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

# Зміна розміру зображень до 32x32 для використання з VGG16
x_train = tf.image.resize(x_train, (32, 32))
x_val = tf.image.resize(x_val, (32, 32))
x_test = tf.image.resize(x_test, (32, 32))

# Перетворення на трьохканальні зображення
x_train = tf.image.grayscale_to_rgb(x_train)
x_val = tf.image.grayscale_to_rgb(x_val)
x_test = tf.image.grayscale_to_rgb(x_test)

4422102/4422102 [==============================] - 0s 0us/step
Train data size (42000, 28, 28)
Test data size (12000, 28, 28)
Validation data size (6000, 28, 28)


In [ ]:
# Завантажити модель VGG16 без верхніх шарів
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(32, 32, 3)))
base_model.trainable = False

# Розморозити верхні шари VGG16
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Побудова моделі
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax'),
])

# Компільовання моделі
model.compile(optimizer=Nadam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Використання Callback-функцій
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization_1 (Bat  (None, 256)               1024      
 chNormalization)                                       

In [ ]:
# Навчання моделі
batch_size = 32
epochs = 20

history = model.fit(x_train, y_train, batch_size=batch_size,
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train) // batch_size,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/20
1312/1312 [==============================] - 552s 417ms/step - loss: 0.6717 - accuracy: 0.7748 - val_loss: 0.4651 - val_accuracy: 0.8333 - lr: 1.0000e-04
Epoch 2/20
1312/1312 [==============================] - 516s 393ms/step - loss: 0.4463 - accuracy: 0.8434 - val_loss: 0.4036 - val_accuracy: 0.8547 - lr: 1.0000e-04
Epoch 3/20
1312/1312 [==============================] - 541s 412ms/step - loss: 0.3949 - accuracy: 0.8597 - val_loss: 0.3830 - val_accuracy: 0.8642 - lr: 1.0000e-04
Epoch 4/20
1312/1312 [==============================] - 541s 412ms/step - loss: 0.3561 - accuracy: 0.8725 - val_loss: 0.3813 - val_accuracy: 0.8635 - lr: 1.0000e-04
Epoch 5/20
1312/1312 [==============================] - 539s 411ms/step - loss: 0.3282 - accuracy: 0.8802 - val_loss: 0.3613 - val_accuracy: 0.8672 - lr: 1.0000e-04
Epoch 6/20
1312/1312 [==============================] - 536s 409ms/step - loss: 0.3063 - accuracy: 0.8897 - val_loss: 0.3596 - val_accuracy: 0.8698 - lr: 1.0000e-04
Epoch 7/20

In [8]:
# Оцінка моделі
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')

Test loss: 0.37660250067710876
Test accuracy: 0.8803333044052124


Багатошарова модель з завдання 1 працює краще.